In [19]:
# Sample dataset
# Format : each row is an example
# The last column is label
# The first two columns are features.
# If you want you can add more features and examples.
# Interesting NOte: 2nd and 5th examples have the same features, but different labels
# Lets see hot tree handle this case.

In [20]:
training_data = [
    ['Green',3,'Mango'],
    ['Yellow',3,'Mango'],
    ['Red',1,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon']
]

training_data

[['Green', 3, 'Mango'],
 ['Yellow', 3, 'Mango'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [21]:
#Adding Column labels.
# These are used only to print the tree.
header = ["color","diameter","label"]

In [22]:
# To find the unique values for a column in a dataset
def unique_vals(rows,col):
    return set([row[col] for row in rows])

In [23]:
#Count the number of each type of example in a dataset

def class_counts(rows):
    counts = {} # a directory of label --> count.
    for row in rows:
        label = row[-1] #in our dataset format, the label is in the last column
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [24]:
# Test us a value is numeric

def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float) 
    #The isinstance() function returns True if the specified object is of the specified type

In [25]:
#Question is used to partition a dataset.

class Question:
    '''This class just records a 'column number'(e.g: 0 for color) and a 'column value' (e.g : Green).
        The 'match' method is used to compare the feature value in an example to the feature value stored in the 
        question.'''
    
    def __init__(self,column,value):
        self.column = column
        self.value = value
    
    def match(self,example):
        #compare the feature value in an example to the feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
        #This is just a helper methos to print the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition,str(self.value))

In [26]:
"""Partition a dataset
    For each row in the dataset, check if it matches the question. 
    if so add it to 'true rows', otherwise, add it to 'false rows'."""

def partition(rows,question):
    true_rows,false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

######
#Demo:
# Lets partition the training data based on whether rows are Red.
#true_rows,false_rows = partition(training_data, Question(0,'Red'))
# true_rows ----> contains all the "Red" rows
# false_rows ----> contains everything else.

In [27]:
""" Calculate the GINI Impurity for a list of rows"""

def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [28]:
""" Information Gain"""

def info_gain(left,right,current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1-p)*gini(right)

#calculate the uncertainty of our training data.
#current_uncertainty = gini(training_data)

#Example .........
#How much info do we gain by partioning on "Green"?
#true_rows, false_rows = partition(training_data, Question(0, "Green"))
#info_gain(true_rows,false_rows,current_uncertainty)

In [45]:
""" Find the best question to ask by iterating over every feature / value and calculating the info gain"""

def find_best_split(rows):
    best_gain = 0 #keep track of the best info gain
    best_question = None #keep train of the feature / value that product it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 #number of columns
    
    for col in range(n_features): #for each feature
        values = set([row[col] for row in rows]) #unique values in the column
        
        for val in values: #for each value
            question = Question(col, val)
            
            #try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            
            #skip this split if it doesnt divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
                
            #calculate the information gain from this split
            gain = info_gain(true_rows,false_rows,current_uncertainty)
            
            #you actually can use '>' instead of '>=' here but
            #i wanted the tree to look a certain way for our toy dataset.
            
            if gain>= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

In [53]:
""" A Leaf node classifies data"""
#This Holds a dicionary of class (e.g : "Mango") --> number of times it appears in the rows from training data that reach this leaf

class Leaf:
    def __init__(self,rows):
        self.predictions = class_counts(rows)

class Decision_Node:
    #Decision node asks a question.
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [54]:
""" Build the tree"""

def build_tree(rows):
    #Try partitioning the dataset on each of the unique attributes,
    #calculate the info gain
    #and return the question that produces the highest gain.
    gain,question = find_best_split(rows)
    
    #Base case: no furthur info gain
    #Since we can ask no furthur question,
    #we'll retuen a leaf
    
    if gain == 0:
        return Leaf(rows)
    
    #if we reach here, we have found a useful feature / value to partition on.
    true_rows, false_rows = partition(rows, question)
    
    # Recursively build the true branch.
    true_branch = build_tree(true_rows)
    
    # Recursively build the false branch.
    false_branch = build_tree(false_rows)
    
    #Return a Question node.
    #This records the best feature / value to ask at this point,
    #as well as the branches to follow
    #depending on the answer
    
    return Decision_Node(question, true_branch, false_branch)

In [55]:
#Worlds most elegant tree printing function

def print_tree(node, spacing=""):
    if isinstance(node,Leaf):
        print(spacing + "Predict", node.predictions)
        return
    #print the question at this node
    print(spacing + str(node.question))
    
    #call this function recursively on the true branch
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + " ")
    
    #call this function recursively on the false branch
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + " ")

In [56]:
def classify(row, node):
    
    #Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    
    #Decide whether to follow the true-branch or the false-branch,
    #Compare the feature / Vlue stored in the node,
    #to the example we're considering.
    
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)

In [57]:
def print_leaf(counts):
    #Print the prediction at a leaf.
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [58]:
testing_data = [
    ['Green',3,'Mango'],
    ['Yellow',4,'Mango'],
    ['Red',2,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon']
]

In [59]:
if __name__ == '__main__':
    my_tree = build_tree(training_data)
    print(my_tree)

for row in testing_data:
    print("Actual: %s. Predicted: %s" % (row[-1], print_leaf(classify(row, my_tree))))

Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
